# Data Preparation

### Settings/Functions
Read in settings and functions.

In [73]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sb
import scipy as sp
import sys
import warnings
from scipy import stats
from scipy.stats import mstats
warnings.filterwarnings("ignore", category=RuntimeWarning)

### Data
Read in the final data set from the data understanding notebook.

In [74]:
# Import original dataset
data_loc = 'C://Users/User/Documents/GitHub/Python/Ohio Reading/data/'
data = pd.read_csv(data_loc+'reading_prepped.csv')

## More Cleaning

### Duplicates

 * No duplicate column names exist in the dataset.

In [75]:
# Checking for duplicate column names
print("data shape: {}".format(data.shape))
data = data.loc[:,~data.columns.duplicated()]
print("data shape: {}".format(data.shape))

data shape: (584, 31)
data shape: (584, 31)


 * All variable names are in a clean, consistent format that is compatible for modeling.

### Variable Names

In [76]:
# Checking variable names
for col in data.columns: 
    print(col) 

year
school
subject
number_tested
pct_prof_or_above
pct_adv
pct_acc
pct_prof
pct_bsc
pct_ltd
tot_pop
perc_male
perc_female
perc_under5
perc_5to19
perc_20plus
perc_white
perc_black
perc_asian
perc_other
perc_nobach
perc_bachplus
perc_employ
perc_unemploy
median_inc
perc_assist
perc_noassist
gini
perc_poverty
perc_nopoverty
time_type


### Categorical Encoding

In [77]:
# Categorical Encoding
print("data types: {}".format(data.dtypes.value_counts()))

data types: float64    24
int64       5
object      2
dtype: int64


In [78]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 584 entries, 0 to 583
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   year               584 non-null    int64  
 1   school             584 non-null    object 
 2   subject            584 non-null    object 
 3   number_tested      584 non-null    int64  
 4   pct_prof_or_above  584 non-null    float64
 5   pct_adv            584 non-null    float64
 6   pct_acc            584 non-null    float64
 7   pct_prof           584 non-null    float64
 8   pct_bsc            584 non-null    float64
 9   pct_ltd            584 non-null    float64
 10  tot_pop            584 non-null    int64  
 11  perc_male          584 non-null    float64
 12  perc_female        584 non-null    float64
 13  perc_under5        584 non-null    float64
 14  perc_5to19         584 non-null    float64
 15  perc_20plus        584 non-null    float64
 16  perc_white         455 non

In [79]:
# Converting 'year' to a factor again
data['year'] = data['year'].astype(object)
print(data.dtypes) 

year                  object
school                object
subject               object
number_tested          int64
pct_prof_or_above    float64
pct_adv              float64
pct_acc              float64
pct_prof             float64
pct_bsc              float64
pct_ltd              float64
tot_pop                int64
perc_male            float64
perc_female          float64
perc_under5          float64
perc_5to19           float64
perc_20plus          float64
perc_white           float64
perc_black           float64
perc_asian           float64
perc_other           float64
perc_nobach          float64
perc_bachplus        float64
perc_employ          float64
perc_unemploy        float64
median_inc             int64
perc_assist          float64
perc_noassist        float64
gini                 float64
perc_poverty         float64
perc_nopoverty       float64
time_type              int64
dtype: object


### Missing Data

 * As discussed in the previous notebook, the race/ethnicity variables will need to be removed due to the shear number of missing values. The remaining variables have very few missing variables so they will be imputed. For this project, I'll just be using the mean for the school district.
 * In this dataset, each school district appears for all four years (2007,2008,2012,2013). Since the education level variables only had 2 missing variables, I knew that I would have data to impute the mean. For the employment data though, I had six missing observations and needed to make sure that one school district was not missing all of its employment data. Once this was verified, the employment data was imputed using the mean for the school district as well.

In [80]:
# Function for counting nulls
def null_counts(df):
    null_df = pd.DataFrame(df.isnull().sum(),columns=['null_count'])
    null_df['null_fraction'] = null_df['null_count'] / df.shape[0]
    null_df = null_df.sort_values('null_count',ascending=False)
    return null_df

In [81]:
# Checking for null values
null_counts(data)

,null_count,null_fraction
perc_white,129,0.220890
perc_other,129,0.220890
perc_asian,129,0.220890
perc_black,129,0.220890
perc_unemploy,6,0.010274
perc_employ,6,0.010274
perc_bachplus,2,0.003425
perc_nobach,2,0.003425
year,0,0.000000
perc_nopoverty,0,0.000000


In [82]:
data = data.drop(['perc_white','perc_other','perc_asian','perc_black'], axis = 1) 
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 584 entries, 0 to 583
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   year               584 non-null    object 
 1   school             584 non-null    object 
 2   subject            584 non-null    object 
 3   number_tested      584 non-null    int64  
 4   pct_prof_or_above  584 non-null    float64
 5   pct_adv            584 non-null    float64
 6   pct_acc            584 non-null    float64
 7   pct_prof           584 non-null    float64
 8   pct_bsc            584 non-null    float64
 9   pct_ltd            584 non-null    float64
 10  tot_pop            584 non-null    int64  
 11  perc_male          584 non-null    float64
 12  perc_female        584 non-null    float64
 13  perc_under5        584 non-null    float64
 14  perc_5to19         584 non-null    float64
 15  perc_20plus        584 non-null    float64
 16  perc_nobach        582 non

In [83]:
# Imputing missing values by group for education values
data['perc_nobach'] = data['perc_nobach'].fillna(data.groupby('school')['perc_nobach'].transform('mean'))
data['perc_bachplus'] = data['perc_bachplus'].fillna(data.groupby('school')['perc_bachplus'].transform('mean'))

In [84]:
NA = data[data['perc_employ'].isnull()]
NA.head

<bound method NDFrame.head of      year                                             school  subject  \
81   2007       New Albany-Plain Local School District, Ohio  Reading   
166  2008                Chardon Local School District, Ohio  Reading   
199  2008                  Kings Local School District, Ohio  Reading   
227  2008       New Albany-Plain Local School District, Ohio  Reading   
235  2008  Northeastern Local School District (Champaign ...  Reading   
274  2008               Twinsburg City School District, Ohio  Reading   

     number_tested  pct_prof_or_above  pct_adv  pct_acc  pct_prof  pct_bsc  \
81             291               92.1     52.2     28.9      11.0      4.8   
166            231               92.2     40.7     41.1      10.4      5.6   
199            286               87.8     33.2     40.9      13.6      8.0   
227            338               91.4     34.3     40.8      16.3      6.2   
235            279               68.8     19.7     30.8      18.3   

In [85]:
NA2 = data[data['perc_unemploy'].isnull()]
NA2.head

<bound method NDFrame.head of      year                                             school  subject  \
81   2007       New Albany-Plain Local School District, Ohio  Reading   
166  2008                Chardon Local School District, Ohio  Reading   
199  2008                  Kings Local School District, Ohio  Reading   
227  2008       New Albany-Plain Local School District, Ohio  Reading   
235  2008  Northeastern Local School District (Champaign ...  Reading   
274  2008               Twinsburg City School District, Ohio  Reading   

     number_tested  pct_prof_or_above  pct_adv  pct_acc  pct_prof  pct_bsc  \
81             291               92.1     52.2     28.9      11.0      4.8   
166            231               92.2     40.7     41.1      10.4      5.6   
199            286               87.8     33.2     40.9      13.6      8.0   
227            338               91.4     34.3     40.8      16.3      6.2   
235            279               68.8     19.7     30.8      18.3   

In [86]:
# Imputing missing values by group for employment data
data['perc_employ'] = data['perc_employ'].fillna(data.groupby('school')['perc_employ'].transform('mean'))
data['perc_unemploy'] = data['perc_unemploy'].fillna(data.groupby('school')['perc_unemploy'].transform('mean'))

In [87]:
# Last check for null values
null_counts(data)

,null_count,null_fraction
year,0,0.0
perc_5to19,0,0.0
perc_nopoverty,0,0.0
perc_poverty,0,0.0
gini,0,0.0
perc_noassist,0,0.0
perc_assist,0,0.0
median_inc,0,0.0
perc_unemploy,0,0.0
perc_employ,0,0.0


### Outliers

 * Having looked at the z-scores in the previous notebook, I have decided to allow outliers to remain in the dataset for the time being. The large majority of the outliers would not necessarily be categorized as extreme.
 * Winsorization: Due to the nature of this dataset, which includes many variables that are meant to add up to 1, it does not make sense to winsorize one variable and not another, as this would mean the variables would no longer add to 1. For example, the variable 'perc_male' has some outliers according to z-scores but 'perc_female' does not. If I were to winsorize the observations for 'perc_male' then those school districts 'perc_male' & 'perc_female' variables would no longer equal 1.
 * Dropping Outliers: Due to the small size of the dataset (only 584 observations), I do not believe that it is in my best interest to remove rows with outliers. Since each school district needs to have observations both before and after the program began, if I were to remove some rows with outliers, some school districts would no longer have the necessary number of observations to be included in the model. 

### Additional Cleaning

In [88]:
# No additional cleaning was performed in this notebook

## Save the Modeling Dataset

In [89]:
# Exporting the data set to CSV 
data.to_csv((data_loc+'/reading_clean.csv'), index = False)

## Outcome

##### The following changes were made to the data set:
 - Categorical Encoding: The variable 'year' was converted to a factor.
 - Missing Data: The missing data was cleaned in the following manner:
      1. All race/ethnicity variables were removed from the dataset due to a large number of missing values.
      2. Remaining missing values were imputed using the mean for their school district.
 - Outliers: Outliers were left in the dataset for the time being. If they cause problems during the modeling notebook then they will be handled accordingly.